In [20]:
class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name
        
    def __get__(self, instance, instance_type):
        if instance is None:
            print(f'{instance} is None')
            return self
        return getattr(instance, self.internal_name, 'haha')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
        
class Customer:
    # Class attributes
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

In [21]:
cust = Customer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')

Before: 'haha' {}


In [22]:
cust.first_name = 'Euclid'
print(f'After: {cust.first_name!r} {cust.__dict__}')

After: 'Euclid' {'_first_name': 'Euclid'}


The problem is that the order of operations in the Custormer class defintion is opposite of how it reads form left to right.

1. the Field constructor is called as Field('first_name')
2. the return value of that is assigned to Customer.field_name.

There is no way for a Field instance to knoe upfront which class attribute it will be assigned to

In [23]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            print('my key:',key,', my value:',value)
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

class DatabaseRow(metaclass=Meta):
    pass

my key: __module__ , my value: __main__
my key: __qualname__ , my value: DatabaseRow


In [24]:
class Field:
    def __init__(self):
        # These will be assigned by the metaclass.
        self.name = None
        self.internal_name = None
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
        
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

my key: __module__ , my value: __main__
my key: __qualname__ , my value: BetterCustomer
my key: first_name , my value: <__main__.Field object at 0x0000017A03A75E70>
my key: last_name , my value: <__main__.Field object at 0x0000017A03A762C0>
my key: prefix , my value: <__main__.Field object at 0x0000017A03A76770>
my key: suffix , my value: <__main__.Field object at 0x0000017A03A77F40>


In [25]:
cust = BetterCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euler'
print(f'After: {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After: 'Euler' {'_first_name': 'Euler'}


In [14]:
class BrokenCustomer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()
    
cust = BrokenCustomer()
cust.first_name = 'Mersenne'

TypeError: attribute name must be string, not 'NoneType'

The solution to this problem is to use the \_\_set\_name\_\_ special method for descriptors.

* This method is called on every descriptors instance when its containing class is defined

In [32]:
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None
        
    def __set_name__(self, owner, name):
        #called on class creation for each descriptor
        print('call')
        self.name = name
        self.internal_name = '_' + name
         
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
        
class FixedCustomer:
    first_name = Field()
    print('first_name fin')
    last_name = Field()
    prefix = Field()
    suffix = Field()
    print('suffix fin')

first_name fin
suffix fin
call
call
call
call


In [27]:
cust = FixedCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Mersenne'
print(f'After: {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After: 'Mersenne' {'_first_name': 'Mersenne'}


In [34]:
cust1 = FixedCustomer()
print(f'Before: {cust1.first_name!r} {cust1.__dict__}')
cust1.first_name = 'Mersenne111'
print(f'After: {cust1.first_name!r} {cust1.__dict__}')

Before: '' {}
After: 'Mersenne111' {'_first_name': 'Mersenne111'}
